# Session 6: Environmental Justice & Vector Data

**Goal:** Analyze the environmental justice implications of data center siting using real demographic and historical data.

Datasets:
1.  **Data Centers**: Locations from Cloud Regions.
2.  **Social Vulnerability Index (SVI)**: CDC's ranking of community vulnerability (Census Tracts).
3.  **Redlining (HOLC)**: Historical maps grading neighborhoods for mortgage risk.

Key concepts:
1.  **Spatial Joins**: Linking indices to infrastructure.
2.  **Historical Context**: Overlapping past policies with current infrastructure.

### Action Item 1: Setup Environment

> **Prompt your Agent:**
> "Import ibis, pandas, anymap, geopandas, and altair. Connect to a local DuckDB instance and load the spatial extension."

In [1]:
import subprocess
import sys

# Install required widget dependencies for anymap (if needed)
subprocess.check_call([sys.executable, "-m", "pip", "install", "anymap", "anywidget", "ipywidgets", "altair", "-q"])

import ibis
import pandas as pd
import geopandas as gpd
import altair as alt
from anymap import MapLibreMap

# Connect to DuckDB and load spatial extension
con = ibis.duckdb.connect()
con.raw_sql("INSTALL spatial; LOAD spatial;")

# Enable interactive mode for Ibis
ibis.options.interactive = True

# Disable ibis progress bar (FloatProgress widget causes VS Code rendering errors)
ibis.options.verbose = False
try:
    con.raw_sql("SET enable_progress_bar = false;")
except Exception:
    pass

print("✓ All libraries imported and DuckDB spatial extension loaded")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✓ All libraries imported and DuckDB spatial extension loaded


## 1. Load Data

We need to load three datasets from the cloud.

**URLs**:
- Data Centers (CSV): `https://s3-west.nrp-nautilus.io/public-datacenters/data_centers.csv`
- SVI 2020 (Parquet): `https://s3-west.nrp-nautilus.io/public-social-vulnerability/svi2020_us_tract.parquet`
- Redlining (Parquet): `https://s3-west.nrp-nautilus.io/public-mappinginequality/mappinginequality.parquet`

### Action Item 2: Load Datasets

> **Prompt your Agent:**
> "Load the Data Centers CSV, SVI Parquet, and Redlining Parquet files into Ibis tables using the provided URLs. For Data Centers, filter for 'United States' and create a geometry column from longitude/latitude. For SVI, filter out any rows where `RPL_THEMES` is negative (missing data)."

In [2]:
from ibis import _

# URLs
url_dcs = "https://s3-west.nrp-nautilus.io/public-datacenters/data_centers.csv"
url_svi = "https://s3-west.nrp-nautilus.io/public-social-vulnerability/svi2020_us_tract.parquet"
url_holc = "https://s3-west.nrp-nautilus.io/public-mappinginequality/mappinginequality.parquet"

# Data Centers (CSV)
dcs_raw = con.read_csv(url_dcs)
dcs_us = (
    dcs_raw
    .mutate(
        latitude_float=_.latitude.cast("float"),
        geom=_.longitude.point(_.latitude.cast("float"))
    )
    .filter(_.country == "United States")
)

# SVI (Parquet) – drop missing values
svi = con.sql(f"SELECT * FROM read_parquet('{url_svi}')")
svi = svi.filter(_.RPL_THEMES >= 0)

# Redlining (Parquet)
holc = con.sql(f"SELECT * FROM read_parquet('{url_holc}')")

# Quick summary (avoid rendering geometry tables)
{
    "dcs_us_rows": dcs_us.count().execute(),
    "svi_rows": svi.count().execute(),
    "holc_rows": holc.count().execute(),
    "dcs_us_schema": str(dcs_us.schema()),
    "svi_schema": str(svi.schema()),
    "holc_schema": str(holc.schema())
}

{'dcs_us_rows': 292,
 'svi_rows': 83331,
 'holc_rows': 10154,
 'dcs_us_schema': 'ibis.Schema {\n  provider        string\n  region_name     string\n  type            string\n  metro           string\n  country         string\n  latitude        string\n  longitude       float64\n  zones           int64\n  latitude_float  float64\n  geom            point:geometry\n}',
 'svi_schema': 'ibis.Schema {\n  OBJECTID      int64\n  ST            string\n  STATE         string\n  ST_ABBR       string\n  STCNTY        string\n  COUNTY        string\n  FIPS          string\n  LOCATION      string\n  AREA_SQMI     float64\n  E_TOTPOP      int32\n  M_TOTPOP      int32\n  E_HU          int32\n  M_HU          int32\n  E_HH          int32\n  M_HH          int32\n  E_POV150      int32\n  M_POV150      int32\n  E_UNEMP       int32\n  M_UNEMP       int32\n  E_HBURD       int32\n  M_HBURD       int32\n  E_NOHSDP      int32\n  M_NOHSDP      int32\n  E_UNINSUR     int32\n  M_UNINSUR     int32\n  E_AGE65       

## 2. Visualize SVI Layers

The SVI dataset contains a ranking `RPL_THEMES` (0 to 1), where higher values indicate higher vulnerability.

**PMTiles URLs**:
- SVI: `https://s3-west.nrp-nautilus.io/public-social-vulnerability/svi2020_us_tract.pmtiles`, layer name is SVI2000_US_tract
- Redlining: `https://s3-west.nrp-nautilus.io/public-mappinginequality/mappinginequality.pmtiles` layer name is redlining

### Action Item 3: Visualize with AnyMap

> **Prompt your Agent:**
> "Create an interactive map using `anymap`. Add the SVI PMTiles as a fill layer (color by `RPL_THEMES`), the Redlining PMTiles as a fill layer (color by `grade`), and the Data Centers as red points. Save the map as '02-ej-analysis-map.html'."

### Debug: Check PMTiles Layer Names

Let's verify the correct source-layer names in the PMTiles files.

In [4]:
# Let's try a simple test map with different source-layer names
# Common PMTiles source-layer names are often just the filename or a simplified version

test_svi = MapLibreMap(center=[-98, 39], zoom=4, height="400px", style=local_style)
test_svi.add_basemap("OpenStreetMap.Mapnik")

# Try with the most common source-layer name pattern: just the base name
test_svi.add_pmtiles(
    svi_pmtiles,
    layer_id="svi_test",
    layers=[
        {
            "id": "svi-test-fill",
            "source": "svi_test",
            "source-layer": "svi2000_us_tract",  # Corrected to match file
            "type": "fill",
            "paint": {
                "fill-color": "#08519c",
                "fill-opacity": 0.5
            }
        }
    ]
)

print("Testing SVI PMTiles with source-layer: svi2000_us_tract")
test_svi

Testing SVI PMTiles with source-layer: svi2020_us_tract


In [5]:
# Convert data centers to GeoDataFrame

dcs_us_df = dcs_us.execute()
dcs_gdf = gpd.GeoDataFrame(dcs_us_df, geometry="geom", crs="EPSG:4326")

# PMTiles sources (note: 2000 in filename, but data is 2020)
svi_pmtiles = "https://s3-west.nrp-nautilus.io/public-social-vulnerability/svi2000_us_tract.pmtiles"
holc_pmtiles = "https://s3-west.nrp-nautilus.io/public-mappinginequality/mappinginequality.pmtiles"

# Create a local style to avoid SSL issues
local_style = {
    "version": 8,
    "sources": {},
    "layers": []
}

# Create map with local style to avoid SSL issues
m = MapLibreMap(center=[-98, 39], zoom=4, height="650px", style=local_style)

# Add OpenStreetMap.Mapnik as base layer
m.add_basemap("OpenStreetMap.Mapnik")

# SVI layer (fill by RPL_THEMES)
m.add_pmtiles(
    svi_pmtiles,
    layer_id="svi",
    layers=[
        {
            "id": "svi-fill",
            "source": "svi",
            "source-layer": "svi2000_us_tract",  # Fixed: matches PMTiles filename
            "type": "fill",
            "paint": {
                "fill-color": [
                    "interpolate",
                    ["linear"],
                    ["get", "RPL_THEMES"],
                    0, "#f7fbff",
                    0.2, "#c6dbef",
                    0.4, "#9ecae1",
                    0.6, "#6baed6",
                    0.8, "#3182bd",
                    1.0, "#08519c"
                ],
                "fill-opacity": 0.6
            }
        }
    ]
)

# Redlining layer (fill by grade)
m.add_pmtiles(
    holc_pmtiles,
    layer_id="holc",
    layers=[
        {
            "id": "holc-fill",
            "source": "holc",
            "source-layer": "redlining",
            "type": "fill",
            "paint": {
                "fill-color": [
                    "match",
                    ["get", "grade"],
                    "A", "#2ECC71",
                    "B", "#F1C40F",
                    "C", "#E67E22",
                    "D", "#E74C3C",
                    "#BDBDBD"
                ],
                "fill-opacity": 0.35
            }
        }
    ]
)

# Data center points
m.add_vector(
    dcs_gdf,
    layer_type="circle",
    paint={
        "circle-radius": 5,
        "circle-color": "#FF0000",
        "circle-opacity": 0.8,
        "circle-stroke-width": 1,
        "circle-stroke-color": "#FFFFFF"
    },
    name="data_centers"
)

# Save map to HTML file
import os
notebook_dir = r"c:\Users\sharo\OneDrive\Desktop\Module-2-Hidden-Figures\notebooks"
html_path = os.path.join(notebook_dir, "ej-analysis-map.html")
m.to_html(html_path, title="EJ Analysis Map")
print(f"Map saved to: {html_path}")
print("Open this file in your web browser to view the map.")

# Also try to display in notebook
m

Map saved to: c:\Users\sharo\OneDrive\Desktop\Module-2-Hidden-Figures\notebooks\ej-analysis-map.html
Open this file in your web browser to view the map.


## 3. Spatial Join: Vulnerability at Data Center Locations

We want to find the SVI score of the tract where each data center is located.

### Action Item 4: Spatial Join & Analysis

> **Prompt your Agent:**
> "Perform a spatial join to find which SVI tract each Data Center falls into. Calculate the average SVI (`RPL_THEMES`) for data centers and compare it to the national average. Create an Altair chart comparing the distribution of SVI scores for Data Centers vs a sample of the national data."

In [7]:
# Spatial join: data centers within SVI tracts
# (SVI geometry column is "Shape")
dcs_svi = dcs_us.join(svi, dcs_us.geom.within(svi.Shape))

# Average SVI for data centers vs national average
avg_dc_svi = dcs_svi.RPL_THEMES.mean().execute()
avg_us_svi = svi.RPL_THEMES.mean().execute()

summary = pd.DataFrame(
    {
        "group": ["Data Centers", "National"],
        "avg_RPL_THEMES": [avg_dc_svi, avg_us_svi]
    }
)

# Sample national SVI distribution for comparison
dc_scores = dcs_svi.select(_.RPL_THEMES).execute()
dc_scores["group"] = "Data Centers"

n_national = 5000 - len(dc_scores)  # stay under Altair's 5000 row limit
svi_sample = svi.select(_.RPL_THEMES).order_by(ibis.random()).limit(n_national)
us_scores = svi_sample.execute()
us_scores["group"] = "National"

scores_df = pd.concat([dc_scores, us_scores], ignore_index=True)

chart = (
    alt.Chart(scores_df)
    .transform_density(
        "RPL_THEMES",
        as_=["RPL_THEMES", "density"],
        groupby=["group"]
    )
    .mark_area(opacity=0.5)
    .encode(
        x=alt.X("RPL_THEMES:Q", title="SVI (RPL_THEMES)"),
        y=alt.Y("density:Q", title="Density"),
        color=alt.Color("group:N", title="Group")
    )
    .properties(width=650, height=350, title="SVI Distribution: Data Centers vs National")
)

from IPython.display import display
display(summary)
chart

127.0.0.1 - - [11/Feb/2026 12:23:09] "GET /02-ej-analysis-map.html HTTP/1.1" 304 -


,group,avg_RPL_THEMES
0,Data Centers,0.535276
1,National,0.499994


alt.Chart(...)

## 4. Redlining Overlay

The HOLC redlining maps graded neighborhoods from 'A' (Best) to 'D' (Hazardous - Red).

### Action Item 5: Redlining Analysis

> **Prompt your Agent:**
> "Perform a spatial join between Data Centers and the Redlining data. Count how many data centers fall into each HOLC grade (A, B, C, D). Create a bar chart visualizing these counts."

In [8]:
# Spatial join: data centers within HOLC redlining zones
holc_renamed = holc.rename(holc_geom="geom")
dcs_holc = dcs_us.join(holc_renamed, dcs_us.geom.within(holc_renamed.holc_geom))

# Get matched data centers with their HOLC grade for the map
dcs_holc_df = dcs_holc.select("provider", "region_name", "metro", "longitude", "latitude_float", "grade", "city", "geom").execute()
dcs_holc_gdf = gpd.GeoDataFrame(dcs_holc_df, geometry="geom", crs="EPSG:4326")

print(f"Found {len(dcs_holc_gdf)} data centers in HOLC zones")
print(f"Total data centers: {len(dcs_gdf)}")

# Map: Data centers on redlining zones
m2 = MapLibreMap(center=[-95, 38], zoom=4, height="650px", style=local_style)
m2.add_basemap("OpenStreetMap.Mapnik")

# Add HOLC redlining layer via PMTiles (complete US coverage)
m2.add_pmtiles(
    "https://s3-west.nrp-nautilus.io/public-mappinginequality/mappinginequality.pmtiles",
    layer_id="holc",
    layers=[
        {
            "id": "holc-fill",
            "source": "holc",
            "source-layer": "redlining",
            "type": "fill",
            "paint": {
                "fill-color": [
                    "match",
                    ["get", "grade"],
                    "A", "#2ECC71",
                    "B", "#F1C40F",
                    "C", "#E67E22",
                    "D", "#E74C3C",
                    "#BDBDBD"
                ],
                "fill-opacity": 0.5
            }
        },
        {
            "id": "holc-outline",
            "source": "holc",
            "source-layer": "redlining",
            "type": "line",
            "paint": {
                "line-color": "#000000",
                "line-width": 1,
                "line-opacity": 0.4
            }
        }
    ]
)

# Add SVI vulnerability layer via PMTiles
m2.add_pmtiles(
    "https://s3-west.nrp-nautilus.io/public-social-vulnerability/svi2020_us_tract.pmtiles",
    layer_id="svi",
    layers=[
        {
            "id": "svi-fill",
            "source": "svi",
            "source-layer": "SVI2000_US_tract",
            "type": "fill",
            "paint": {
                "fill-color": [
                    "interpolate",
                    ["linear"],
                    ["get", "RPL_THEMES"],
                    0, "#f7fbff",
                    0.2, "#c6dbef",
                    0.4, "#9ecae1",
                    0.6, "#6baed6",
                    0.8, "#3182bd",
                    1.0, "#08519c"
                ],
                "fill-opacity": 0.3
            }
        }
    ]
)

# All data centers (smaller, red)
m2.add_vector(
    dcs_gdf,
    layer_type="circle",
    paint={
        "circle-radius": 5,
        "circle-color": "#FF0000",
        "circle-opacity": 0.8,
        "circle-stroke-width": 1,
        "circle-stroke-color": "#FFFFFF"
    },
    name="all_data_centers"
)

# Data centers that fall IN redlined zones (yellow, larger)
if len(dcs_holc_gdf) > 0:
    m2.add_vector(
        dcs_holc_gdf,
        layer_type="circle",
        paint={
            "circle-radius": 8,
            "circle-color": "#FFFF00",
            "circle-opacity": 0.95,
            "circle-stroke-width": 2,
            "circle-stroke-color": "#000000"
        },
        name="dc_in_holc"
    )

# Save to HTML
m2.to_html("02-ej-redlining-map.html", title="Data Centers & Redlining")

# Display in IFrame
from IPython.display import IFrame
IFrame("http://localhost:8000/02-ej-redlining-map.html", width="100%", height=650)

Found 128 data centers in HOLC zones
Total data centers: 292


127.0.0.1 - - [11/Feb/2026 12:23:50] "GET /02-ej-redlining-map.html HTTP/1.1" 200 -


In [ ]:
# DEBUG: Test just the vector points first
test_map = MapLibreMap(center=[-95, 38], zoom=4, height="500px", style=local_style)
test_map.add_basemap("OpenStreetMap.Mapnik")

# Just add the data center points
test_map.add_vector(
    dcs_gdf,
    layer_type="circle",
    paint={
        "circle-radius": 6,
        "circle-color": "#FF0000",
        "circle-opacity": 0.9
    },
    name="test_points"
)

test_map.to_html("test-map.html", title="Test Map")
from IPython.display import IFrame
IFrame("http://localhost:8000/test-map.html", width="100%", height=500)

127.0.0.1 - - [09/Feb/2026 14:13:03] "GET /test-map.html HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2026 14:13:14] "GET /test-map.html?id=6f13bd6c-a130-4b6d-b2be-78c0605f2b51&vscodeBrowserReqId=1770675194432 HTTP/1.1" 200 -


In [ ]:
# DEBUG: Test PMTiles redlining layer by itself
test_pmtiles = MapLibreMap(center=[-74.0, 40.7], zoom=11, height="500px", style=local_style)  # NYC zoom
test_pmtiles.add_basemap("OpenStreetMap.Mapnik")

test_pmtiles.add_pmtiles(
    "https://s3-west.nrp-nautilus.io/public-mappinginequality/mappinginequality.pmtiles",
    layer_id="test_holc",
    layers=[
        {
            "id": "test-holc-fill",
            "source": "test_holc",
            "source-layer": "redlining",
            "type": "fill",
            "paint": {
                "fill-color": "#E74C3C",
                "fill-opacity": 0.6
            }
        }
    ]
)

test_pmtiles.to_html("test-pmtiles.html", title="Test PMTiles")
from IPython.display import IFrame
IFrame("http://localhost:8000/test-pmtiles.html", width="100%", height=500)

In [9]:
# Count data centers by HOLC grade
grade_counts = (
    dcs_holc
    .group_by("grade")
    .agg(count=_.grade.count())
    .order_by("grade")
    .execute()
)

holc_colors = {"A": "#2ECC71", "B": "#F1C40F", "C": "#E67E22", "D": "#E74C3C"}

from IPython.display import display
display(grade_counts)

alt.Chart(grade_counts).mark_bar().encode(
    x=alt.X("grade:N", title="HOLC Grade", sort=["A", "B", "C", "D"]),
    y=alt.Y("count:Q", title="Number of Data Centers"),
    color=alt.Color("grade:N", scale=alt.Scale(domain=list(holc_colors.keys()), range=list(holc_colors.values())), legend=None)
).properties(width=400, height=350, title="Data Centers in Historically Redlined Neighborhoods")

,grade,count
0,C,2
1,D,22
2,None,0


alt.Chart(...)

## Discussion

How does the geography of the cloud align with the geography of inequality? (Note: Many data centers are in newer suburbs (e.g. Ashburn) that post-date redlining maps, so overlaps might be limited to urban interconnection hubs).